In [1]:
!pip install influxdb-client

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.5/65.5 kB 1.2 MB/s eta 0:00:00a 0:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 746.3/746.3 kB 5.1 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 217.8/217.8 kB 3.9 MB/s eta 0:00:0000:01

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [8]:
# Step 2: Import libraries
from influxdb_client import InfluxDBClient
import pandas as pd

# Step 3: Define connection parameters
url = "http://192.168.178.78:8086/"
token = "aFqHDK6EKcy6HSgIBYUAXUGLV4Fh5HCxmbkKJB8enwbDUnJBNTLj_68KCm4HvKk5oyBByzORR55DeHwOyKwgww=="
org = "microfactory"
bucket = "microfactory-jupiter-DS"

# Step 4: Create InfluxDB client
client = InfluxDBClient(url=url, token=token, org=org)
query_api = client.query_api()

# Step 5: Flux query to get unique machines
query = f'''
from(bucket: "{bucket}")
  |> range(start: 1970-01-01T00:00:00Z)
  |> filter(fn: (r) => r["_measurement"] == "machine")
  |> keep(columns: ["brand", "type", "device"])
  |> group(columns: ["brand", "type", "device"])
  |> distinct(column: "device")
'''

# Step 6: Execute query and extract records
tables = query_api.query(query)

records = []
for table in tables:
    for record in table.records:
        records.append({
            "brand": record.values.get("brand"),
            "type": record.values.get("type"),
            "device": record.values.get("device")
        })

# Step 7: Convert to Pandas DataFrame and remove duplicates
df_unique_machines = pd.DataFrame(records).drop_duplicates().reset_index(drop=True)

# Step 8: Map to new schema
df_mapped = pd.DataFrame({
    "equipment_id": [None] * len(df_unique_machines),  # empty
    "resource_id": [None] * len(df_unique_machines),   # empty
    "brand": df_unique_machines["brand"],
    "model": df_unique_machines["device"],             # map 'device' to 'model'
    "type": df_unique_machines["type"],               # keep 'type'
    "description": [None] * len(df_unique_machines),  # empty
    "source_id": [None] * len(df_unique_machines)     # empty
})

# Step 9: Display final DataFrame
df_mapped



[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


,equipment_id,resource_id,brand,model,type,description,source_id
0,None,None,prusa,mk4-1,None,None,None
1,None,None,prusa,prusa-mk4-1,3dp,None,None
